In [2]:
import sys
sys.path.append("../../models/PFN")
from dataset_pfn import PFNDataset
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from pfn_model import ParticleFlowNetwork as Model
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score
import os, json

In [4]:
all_models = [f for f in os.listdir("../../models/PFN/trained_models") if "_best" in f]
print("\n".join(all_models))

PFN_best_latent_128
PFN_best_modified_v7
PFN_best_modified_v4
PFN_best_latent_64
PFN_best


In [8]:
test_path = "../../datasets/test.h5"

#Loading testing dataset
test_set = PFNDataset(test_path, preprocessed=True)
testloader = DataLoader(test_set, shuffle=False, batch_size=500, num_workers=30, pin_memory=True, persistent_workers=True)

In [11]:
def eval2(model):
    labels = []
    preds = []
    with torch.no_grad():
        for x,m,y,_ in tqdm(testloader):
            x = x.cuda()
            m = m.cuda()
            pred = model(x, m)
            labels.append(y[:,1].cpu().numpy())
            preds.append(pred[:,1].cpu().numpy())
    labels = np.concatenate(labels, axis=None)
    preds = np.concatenate(preds, axis=None)
    return labels, preds

In [12]:
#loading model
for modelname in all_models:
    model_dict = json.load(open("../../models/PFN/trained_model_dicts/" + modelname.replace("_best","") + ".json"))
    #print(modelname, model_dict)
    label = model_dict['label']
    f_nodes = list(map(int, model_dict['f_nodes'].split(',')))
    phi_nodes = list(map(int, model_dict['phi_nodes'].split(',')))

    model = Model(input_dims=3, Phi_sizes=phi_nodes, F_sizes=f_nodes).cuda()
    model.load_state_dict(torch.load("../../models/PFN/trained_models/" + modelname ))
    labels, preds = eval2(model)
    accuracy = accuracy_score(labels, preds.round())*100
    auc = roc_auc_score(labels, preds)*100
    print(modelname, "\t", "ROC-AUC: {:.4f}% Accuracy: {:.4f}%".format(auc, accuracy))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 808/808 [00:08<00:00, 95.18it/s]


PFN_best_latent_128 	 ROC-AUC: 99.7206% Accuracy: 97.5767%


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 808/808 [00:02<00:00, 384.66it/s]


PFN_best_modified_v7 	 ROC-AUC: 99.4146% Accuracy: 96.8923%


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 808/808 [00:01<00:00, 405.01it/s]


PFN_best_modified_v4 	 ROC-AUC: 99.1424% Accuracy: 96.2740%


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 808/808 [00:02<00:00, 399.82it/s]


PFN_best_latent_64 	 ROC-AUC: 99.4225% Accuracy: 97.1037%


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 808/808 [00:02<00:00, 397.02it/s]


PFN_best 	 ROC-AUC: 99.7162% Accuracy: 97.7178%
